In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [3]:
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')
batch_size = 16
epoch = 25

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_set = torchvision.datasets.CIFAR10(root='./data',train=True,
                                        download=True,transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data',train=False,
                                       download=True,transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                         shuffle=False,num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv = nn.Conv2d(3,10,5)
        self.pool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(10*14*14, 10)
        # self.dropout = nn.Dropout(0.2)

    def forward(self,x):
        x = self.pool(torch.nn.functional.relu(self.conv(x)))
        
        x = x.view(-1, 10*14*14)
        # x = self.dropout(x)
        x = torch.nn.functional.relu(self.fc(x))
        # x = self.dropout(x)
        return x

In [8]:
cnn = CNN()
loss_f = nn.CrossEntropyLoss()
for lr in [0.001]:
    optimizer = optim.SGD(cnn.parameters(),lr=lr,momentum=0.9)

    for i in range(epoch):
        train_loss = 0
    
        for data,target in train_loader:
            optimizer.zero_grad()
            output = cnn(data)
            loss = loss_f(output,target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)
    
        train_loss = train_loss/len(train_loader.dataset)
    
        print('lr:{}\t Epoch:{}\t training loss:{:.6f}'.format(lr,i+1, train_loss))
        

lr:0.001	 Epoch:1	 training loss:1.605243
lr:0.001	 Epoch:2	 training loss:1.309145
lr:0.001	 Epoch:3	 training loss:1.202522
lr:0.001	 Epoch:4	 training loss:1.142802
lr:0.001	 Epoch:5	 training loss:1.111117
lr:0.001	 Epoch:6	 training loss:1.085991
lr:0.001	 Epoch:7	 training loss:1.070140
lr:0.001	 Epoch:8	 training loss:1.049443
lr:0.001	 Epoch:9	 training loss:1.037443
lr:0.001	 Epoch:10	 training loss:1.028708
lr:0.001	 Epoch:11	 training loss:1.016233
lr:0.001	 Epoch:12	 training loss:1.009480
lr:0.001	 Epoch:13	 training loss:0.999249
lr:0.001	 Epoch:14	 training loss:0.991132
lr:0.001	 Epoch:15	 training loss:0.985612
lr:0.001	 Epoch:16	 training loss:0.980019
lr:0.001	 Epoch:17	 training loss:0.972679
lr:0.001	 Epoch:18	 training loss:0.968177
lr:0.001	 Epoch:19	 training loss:0.963288
lr:0.001	 Epoch:20	 training loss:0.953977
lr:0.001	 Epoch:21	 training loss:0.950943
lr:0.001	 Epoch:22	 training loss:0.947395
lr:0.001	 Epoch:23	 training loss:0.942043
lr:0.001	 Epoch:24	 

In [9]:
def test_res():
    correct = list(0. for j in range(10))
    total = list(0. for j in range(10))
    with torch.no_grad():
        for images,labels in test_loader:
            outputs = cnn(images)
            _,predicted = torch.max(outputs.data, 1)
            c = (predicted == labels).squeeze()
            for j in range(4):
                label = labels[j]
                correct[label] += c[j].item()
                total[label] += 1
    for j in range(10):
        print('accuracy of %5s :%2d %%'%(
            classes[j], 100 * correct[j] / total[j]))
    
    return 0
test_res()

accuracy of plane :64 %
accuracy of   car :77 %
accuracy of  bird :49 %
accuracy of   cat :47 %
accuracy of  deer :55 %
accuracy of   dog :47 %
accuracy of  frog :68 %
accuracy of horse :64 %
accuracy of  ship :69 %
accuracy of truck :74 %


0

In [ ]:
def test_all():
    correct = 0
    total = 0
    with torch.no_grad():
        for images,labels in test_loader:
            outputs = cnn(images)
            _,predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
    print('test accuracy:%d %%'%( 100 * correct / total ))
    return 100 * correct / total